In [ ]:
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ann import ANNRegressor_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as MAE

Function - Preprocess & feature engineering

In [ ]:
# Set the parameter 'training' to True if the label exists

def preprocess_featureengineering(df, training):
    
    # Identify and handle null items
    if training: # The label is only available in training
        idx_null = np.where(df['Delivery Time (days)'].isnull()==1)[0]
        df.drop(idx_null, axis=0, inplace=True) # Remove the samples without the labels
        df.reset_index(drop=True, inplace=True) # Reset index
    df.fillna('0', inplace=True) # Fill null item with '0' for attribute 'Order Type'
    
    # Feature engineering with attribute 'Order Create Date'
    df['Month'] = df['Order Create Date'].apply(lambda x: x.month)
    df['Day'] = df['Order Create Date'].apply(lambda x: x.day)
    df['Weekend'] = df['Order Create Date'].apply(lambda x: 1 if x.weekday() >=5 else 0)    
    
       
    # One hot encode categorical attributes
    columns_categorical = [['Order ID', 'driver_ID', 
                         'Trip_ID', 'Trip_origin', 'Trip_ditination', 
                         'Trip Start Time', 'Trip End Time']
    df = pd.get_dummies(df, columns=columns_categorical, drop_first=True)
    
    # Create important attributes/ selected features if it is missing from the current dataset
    # Assign value to be zeros
    for selected_feature in selected_features:
        if selected_feature not in df.columns.tolist():
            df[selected_feature] = 0   
    
    return df

Function - Standardization of numerical features


In [ ]:
def standardize(df, training):
    
    # Standardization of delivery time and order product quantity
    x_categorical = df[selected_features].drop('Order_ID', axis=1, inplace=False).values
    x_numerical = df['Driver_ID'].values
    x_numerical = quantity_scaler.transform(np.expand_dims(x_numerical, axis=1))
    x_test = np.concatenate((x_categorical, x_numerical), axis=1)
    
    if training:
        y = df['Delivery Time (days)']
        y_test = deliverytime_scaler.transform(np.expand_dims(y, axis=1))
        # return scaled features/ attributes and labels
        return x_test, y_test
    
    return x_test

In [ ]:
Function - Execution pipeline

In [ ]:
def execution_pipeline(df, training):
    
    df = preprocess_featureengineering(df, training)
    
    # Predict and save results and evaluation metrics
    x_test, y_test = standardize(df, training) # Scaled features & labels
    y_test_pred = model.predict(x_test) # rounded unscaled prediction time in days
    
    # Note that the prediction return is unscaled while the actual needs to be scaled back
    result = pd.DataFrame({'Predicted': np.squeeze(y_test_pred), 
                           'Actual': np.squeeze(deliverytime_scaler.inverse_transform(y_test))})
    print(result.head(), '\n')
    mae, percentage_delayed_delivery = model.evaluate(x_test, y_test, plot=False)
    
    
    with open('result.pkl', 'wb') as file:
        pickle.dump([result, mae, percentage_delayed_delivery], file)    

In [ ]:
model = ANNRegressor_model(units, num_features, quantile, 
                           file_dir, epochs, batch_size,
                           deliverytime_scaler)

In [ ]:
if __name__ == '__main__':
    
    # Read file and preprocess & feature engineer
    df = pd.read_excel('data_test.xlsx')
    
    # Set the parameter 'training' to True if the label exists
    training = True
    
    # Execution pipeline
    execution_pipeline(df, training)